In [13]:
!pip install requests pandas beautifulsoup4


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import pandas as pd


def get_india_news(api_key, query="nifty", language="en", country="in", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      country (str, optional): The country to focus on. Defaults to "in" (India).
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, description, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
      "country": country
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, description, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        description = article["description"] if "description" in article else ""
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "description": description, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("news100.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-22"
end_date = "2024-08-22"
generate_news_file(api_key, start_date, end_date)

An error occurred while fetching news: 400 Client Error: Bad Request for url: https://newsapi.org/v2/everything?apiKey=d1301d016d024dcb819e57ade041fe67&q=nifty&language=en&country=in&sortBy=publishedAt&from=2024-08-22&to=2024-08-22
Error: Failed to retrieve news data.


In [1]:
import requests
import pandas as pd


def get_news(api_key, query="Nifty", language="en",
             country="in", start_date=None, end_date=None):
    """
    Fetches relevant news articles using the Newsdata API (alternative to Google News API).

    Args:
        api_key (str): Your Newsdata API key.
        query (str, optional): The search query. Defaults to "top headlines".
        language (str, optional): The language of the articles. Defaults to "en".
        country (str, optional): The country code (ISO 3166-1 alpha-2) to filter by. Defaults to "in" (India).
        start_date (str, optional): The start date in YYYY-MM-DD format.
        end_date (str, optional): The end date in YYYY-MM-DD format.

    Returns:
        pd.DataFrame: A DataFrame containing the fetched news data (title, description, url, publishedAt),
                      or None if an error occurs.
    """

    url = "https://api.newsdata.io/v1/news"
    params = {
        "apikey": api_key,
        "q": query,
        "language": language,
        "country": country,
    }

    if start_date and end_date:
        params["from"] = start_date
        params["to"] = end_date

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        data = response.json()

        if data["status"] == "ok":
            articles = data["results"]

            # Extract title, description, url, and publishedAt
            news_data = []
            for article in articles:
                title = article["title"]
                description = article["description"] if "description" in article else ""
                url = article["link"]
                publishedAt = article["pubDate"]
                news_data.append({"title": title, "description": description, "url": url, "publishedAt": publishedAt})

            df = pd.DataFrame(news_data)
            return df
        else:
            print(f"Error: {data['message']}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching news: {e}")
        return None


def generate_news_file(api_key, start_date, end_date):
    """
    Generates a CSV file containing news headlines for a specified date range.

    Args:
        api_key (str): Your Newsdata API key.
        start_date (str): The start date in YYYY-MM-DD format.
        end_date (str): The end date in YYYY-MM-DD format.
    """

    news_data = get_news(api_key, start_date=start_date, end_date=end_date)

    if news_data is not None:
        news_data.to_csv("newsdata1.csv", index=False)
    else:
        print("Error: Failed to retrieve news data.")


# Example usage (replace with your actual Newsdata API key)
api_key = "pub_51806f0c9c5bcbf9a670d57826141522e9fe8"  # Replace with your API key
start_date = "2024-07-24"
end_date = "2024-07-25"
generate_news_file(api_key, start_date, end_date)

An error occurred while fetching news: HTTPSConnectionPool(host='api.newsdata.io', port=443): Max retries exceeded with url: /v1/news?apikey=pub_51806f0c9c5bcbf9a670d57826141522e9fe8&q=Nifty&language=en&country=in&from=2024-07-24&to=2024-07-25 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001CE3EFFED50>: Failed to resolve 'api.newsdata.io' ([Errno 11001] getaddrinfo failed)"))
Error: Failed to retrieve news data.


In [16]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("24july.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-07-24"
end_date = "2024-07-24"
generate_news_file(api_key, start_date, end_date)


In [1]:
import requests
import pandas as pd


def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
    """
    Fetches relevant news articles about India using the Google News API.

    Args:
        api_key (str): Your Google News API key.
        query (str, optional): The search query. Defaults to "nifty".
        language (str, optional): The language of the articles. Defaults to "en".
        start_date (str, optional): The start date in YYYY-MM-DD format.
        end_date (str, optional): The end date in YYYY-MM-DD format.

    Returns:
        pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
                      or None if an error occurs.
    """

    url = "https://newsapi.org/v2/everything"
    params = {
        "apiKey": api_key,
        "q": query,
        "language": language,
    }

    if start_date and end_date:
        params["sortBy"] = "publishedAt"
        params["from"] = start_date
        params["to"] = end_date

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        data = response.json()

        if data["status"] == "ok":
            articles = data["articles"]

            # Extract title, url, and publishedAt
            news_data = []
            for article in articles:
                title = article["title"]
                url = article["url"]
                publishedAt = article.get("publishedAt")  # Handle missing 'publishedAt'

                if title and url:  # Only include entries with both title and url
                    news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

            df = pd.DataFrame(news_data)
            return df
        else:
            print(f"API Error: {data['message']}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching news: {e}")
        return None


def generate_news_file(api_key, start_date, end_date):
    """
    Generates a CSV file containing news headlines for a specified date range.

    Args:
        api_key (str): Your Google News API key.
        start_date (str): The start date in YYYY-MM-DD format.
        end_date (str): The end date in YYYY-MM-DD format.
    """

    news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

    if news_data is not None:
        news_data.to_csv("andu.csv", index=False)
    else:
        print("Error: Failed to retrieve news data.")


# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"  # Replace with your Google News API key
start_date = "2024-08-24"
end_date = "2024-08-24"
generate_news_file(api_key, start_date, end_date)

In [2]:
import requests
import pandas as pd


def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
    """
    Fetches relevant news articles about India using the Google News API.

    Args:
        api_key (str): Your Google News API key.
        query (str, optional): The search query. Defaults to "nifty".
        language (str, optional): The language of the articles. Defaults to "en".
        start_date (str, optional): The start date in YYYY-MM-DD format.
        end_date (str, optional): The end date in YYYY-MM-DD format.

    Returns:
        pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt, description),
                      or None if an error occurs.
    """

    url = "https://newsapi.org/v2/everything"
    params = {
        "apiKey": api_key,
        "q": query,
        "language": language,
    }

    if start_date and end_date:
        params["sortBy"] = "publishedAt"
        params["from"] = start_date
        params["to"] = end_date

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        data = response.json()

        if data["status"] == "ok":
            articles = data["articles"]

            # Extract title, url, publishedAt, and description (handle missing values)
            news_data = []
            for article in articles:
                title = article["title"]
                url = article["url"]
                publishedAt = article.get("publishedAt")
                description = article.get("description")  # Get description if available
                if not description:
                    description = ""  # Set an empty string for missing descriptions

                if title and url:  # Only include entries with both title and url
                    news_data.append({"title": title, "url": url, "publishedAt": publishedAt, "description": description})

            df = pd.DataFrame(news_data)
            return df
        else:
            print(f"API Error: {data['message']}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching news: {e}")
        return None


def generate_news_file(api_key, start_date, end_date):
    """
    Generates a CSV file containing news headlines for a specified date range.

    Args:
        api_key (str): Your Google News API key.
        start_date (str): The start date in YYYY-MM-DD format.
        end_date (str): The end date in YYYY-MM-DD format.
    """

    news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

    if news_data is not None:
        news_data.to_csv("news56_data.csv", index=False)
    else:
        print("Error: Failed to retrieve news data.")


# Example usage (replace with your actual API key)
api_key = "75751f93d546429aae0cc595544a9099"  # Replace with your Google News API key
start_date = "2024-07-24"
end_date = "2024-07-24"
generate_news_file(api_key, start_date, end_date)

An error occurred while fetching news: 426 Client Error: Upgrade Required for url: https://newsapi.org/v2/everything?apiKey=75751f93d546429aae0cc595544a9099&q=nifty&language=en&sortBy=publishedAt&from=2024-07-24&to=2024-07-24
Error: Failed to retrieve news data.


In [1]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("2twestuly.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-07-29"
end_date = "2024-07-29"
generate_news_file(api_key, start_date, end_date)

An error occurred while fetching news: 426 Client Error: Upgrade Required for url: https://newsapi.org/v2/everything?apiKey=d1301d016d024dcb819e57ade041fe67&q=nifty&language=en&sortBy=publishedAt&from=2024-07-29&to=2024-07-29
Error: Failed to retrieve news data.


In [18]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("30july.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-07-30"
end_date = "2024-07-30"
generate_news_file(api_key, start_date, end_date)

In [19]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("31july.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-07-31"
end_date = "2024-07-31"
generate_news_file(api_key, start_date, end_date)

In [20]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("1august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-01"
end_date = "2024-08-01"
generate_news_file(api_key, start_date, end_date)

In [21]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("2august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-02"
end_date = "2024-08-02"
generate_news_file(api_key, start_date, end_date)

In [22]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("3august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-03"
end_date = "2024-08-03"
generate_news_file(api_key, start_date, end_date)

In [23]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("4august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-04"
end_date = "2024-08-04"
generate_news_file(api_key, start_date, end_date)

In [24]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("5august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-05"
end_date = "2024-08-05"
generate_news_file(api_key, start_date, end_date)

In [25]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("6august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-06"
end_date = "2024-08-06"
generate_news_file(api_key, start_date, end_date)

In [26]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("7august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-07"
end_date = "2024-08-07"
generate_news_file(api_key, start_date, end_date)

In [27]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("8august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-08"
end_date = "2024-08-08"
generate_news_file(api_key, start_date, end_date)

In [28]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("9august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-09"
end_date = "2024-08-09"
generate_news_file(api_key, start_date, end_date)

In [29]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("10august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-10"
end_date = "2024-08-10"
generate_news_file(api_key, start_date, end_date)

In [30]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("11august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-11"
end_date = "2024-08-11"
generate_news_file(api_key, start_date, end_date)

In [31]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("12august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-12"
end_date = "2024-08-12"
generate_news_file(api_key, start_date, end_date)

In [32]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("13august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-13"
end_date = "2024-08-13"
generate_news_file(api_key, start_date, end_date)

In [33]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("14august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-14"
end_date = "2024-08-14"
generate_news_file(api_key, start_date, end_date)

In [34]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("15august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-15"
end_date = "2024-08-15"
generate_news_file(api_key, start_date, end_date)

In [35]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("16august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-16"
end_date = "2024-08-16"
generate_news_file(api_key, start_date, end_date)

In [36]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("17august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-17"
end_date = "2024-08-17"
generate_news_file(api_key, start_date, end_date)

In [37]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("18august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-18"
end_date = "2024-08-18"
generate_news_file(api_key, start_date, end_date)

In [38]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("19august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-19"
end_date = "2024-08-19"
generate_news_file(api_key, start_date, end_date)

In [39]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("20august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-20"
end_date = "2024-08-20"
generate_news_file(api_key, start_date, end_date)

In [40]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("21august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-21"
end_date = "2024-08-21"
generate_news_file(api_key, start_date, end_date)

In [41]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("22august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-22"
end_date = "2024-08-22"
generate_news_file(api_key, start_date, end_date)

In [42]:
import requests
import pandas as pd

def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
  """
  Fetches relevant news articles about India using the Google News API.

  Args:
      api_key (str): Your Google News API key.
      query (str, optional): The search query. Defaults to "nifty".
      language (str, optional): The language of the articles. Defaults to "en".
      start_date (str, optional): The start date in YYYY-MM-DD format.
      end_date (str, optional): The end date in YYYY-MM-DD format.

  Returns:
      pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
          or None if an error occurs.
  """

  url = "https://newsapi.org/v2/everything"
  params = {
      "apiKey": api_key,
      "q": query,
      "language": language,
  }

  if start_date and end_date:
      params["sortBy"] = "publishedAt"
      params["from"] = start_date
      params["to"] = end_date

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    data = response.json()

    if data["status"] == "ok":
      articles = data["articles"]

      # Extract title, url, and publishedAt
      news_data = []
      for article in articles:
        title = article["title"]
        url = article["url"]
        publishedAt = article["publishedAt"]
        news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

      df = pd.DataFrame(news_data)
      return df
    else:
      print(f"Error: {data['message']}")
      return None  # Indicate error

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching news: {e}")
    return None  # Indicate error

def generate_news_file(api_key, start_date, end_date):
  """
  Generates a CSV file containing news headlines for a specified date range.

  Args:
      api_key (str): Your Google News API key.
      start_date (str): The start date in YYYY-MM-DD format.
      end_date (str): The end date in YYYY-MM-DD format.
  """

  news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

  if news_data is not None:
    news_data.to_csv("23august.csv", index=False)
  else:
    print("Error: Failed to retrieve news data.")

# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-23"
end_date = "2024-08-23"
generate_news_file(api_key, start_date, end_date)

In [41]:
import requests
import pandas as pd


def get_india_news(api_key, query="nifty", language="en", start_date=None, end_date=None):
    """
    Fetches relevant news articles about India using the Google News API.

    Args:
        api_key (str): Your Google News API key.
        query (str, optional): The search query. Defaults to "nifty".
        language (str, optional): The language of the articles. Defaults to "en".
        start_date (str, optional): The start date in YYYY-MM-DD format.
        end_date (str, optional): The end date in YYYY-MM-DD format.

    Returns:
        pd.DataFrame: A DataFrame containing the fetched news data (title, url, publishedAt),
                     or None if an error occurs.
    """

    url = "https://newsapi.org/v2/everything"
    params = {
        "apiKey": api_key,
        "q": query,
        "language": language,
    }

    if start_date and end_date:
        params["sortBy"] = "publishedAt"
        params["from"] = start_date
        params["to"] = end_date

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        data = response.json()

        if data["status"] == "ok":
            articles = data["articles"]

            # Extract title, url, and publishedAt with date formatting
            news_data = []
            for article in articles:
                title = article["title"]
                url = article["url"]
                publishedAt = article["publishedAt"]

                # Extract year, month, and date using slicing
                publishedAt = publishedAt[:10]  # Get first 10 characters (YYYY-MM-DD)

                news_data.append({"title": title, "url": url, "publishedAt": publishedAt})

            df = pd.DataFrame(news_data)
            return df
        else:
            print(f"Error: {data['message']}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching news: {e}")
        return None


def generate_news_file(api_key, start_date, end_date):
    """
    Generates a CSV file containing news headlines for a specified date range.

    Args:
        api_key (str): Your Google News API key.
        start_date (str): The start date in YYYY-MM-DD format.
        end_date (str): The end date in YYYY-MM-DD format.
    """

    news_data = get_india_news(api_key, start_date=start_date, end_date=end_date)

    if news_data is not None:
        news_data.to_csv("3August.csv", index=False)
    else:
        print("Error: Failed to retrieve news data.")


# Example usage (replace with your actual API key)
api_key = "d1301d016d024dcb819e57ade041fe67"
start_date = "2024-08-03"
end_date = "2024-08-03"
generate_news_file(api_key, start_date, end_date)

In [42]:
import pandas as pd

# List of filenames
filenames =['1August.csv',
'2August.csv',
'3August.csv',
'4August.csv',
'5August.csv',
'6August.csv',
'7August.csv',
'8August.csv',
'9August.csv',
'10August.csv',
'11August.csv',
'12August.csv',
'13August.csv',
'14August.csv',
'15August.csv',
'16August.csv',
'17August.csv',
'18August.csv',
'19August.csv',
'20August.csv',
'21August.csv',
'22August.csv',
'23August.csv',
'24August.csv',
'25August.csv',
'26August.csv',
'27August.csv',
'28August.csv',
'29August.csv',
'30August.csv',
'31August.csv']
# Read CSV files and concatenate
df = pd.concat([pd.read_csv(filename) for filename in filenames])

# Save the merged DataFrame to a new CSV file
df.to_csv('merged_data1.csv', index=False)